In [ ]:
# 기본 라이브러리 호출
import warnings
warnings.filterwarnings('ignore')
warnings.warn("once")
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew
# 선형모델을 추정하는 라이브러리
import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels.stats.api as sms
from patsy import dmatrices
color = sns.color_palette()

pd.set_option('display.float_format','{:,.4f}'.format) # 소수점 2번째 자리까지 표현
pd.set_option('display.max_columns', None) # 모든 컬럼 표시
pd.set_option('display.max_colwidth', -1) # 컬럼내용 전체 표시

#Graph에 한글을 표시하기 위한 코드
import matplotlib
from matplotlib import font_manager, rc
import platform
# font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
font_name = font_manager.FontProperties(fname="/home/spa/.local/share/Trash/files/one/malgun.ttf").get_name()
rc('font', family=font_name)\
    
matplotlib.rcParams['axes.unicode_minus'] = False

# 1. 정형 데이터마이닝 (사용 데이터 : weatherAUS)

### Q1) 데이터의 요약값을 보고 NA값이 10,000개 이상인 열을 제외하고 남은 변수 중 NA값이 있는 행을 제거하시오. 
###     그리고 AUS 데이터의 Date 변수를 Date형으로 변환하고, 
###     전처리가 완료된 weatherAUS 데이터를 train(70%), test(30%) 데이터로 분할하시오.
###     (set.seed(6789)를 실행한 후 데이터를 분할하시오.)

In [ ]:
df = pd.read_csv('./data/weatherAUS.csv')
print(df.shape)
df.head()

In [ ]:
df.isna().sum()

In [ ]:
drop_columns = ['WindDir9am', 'Pressure9am', 'Pressure3pm', "Cloud9am", "Cloud3pm"]
df = df.drop(columns = drop_columns)

In [ ]:
df.isna().sum()

In [ ]:
print(len(df))
print(len(df.dropna()))

In [ ]:
df = df.dropna(axis = 0)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df.head()

In [ ]:
X = pd.get_dummies(df.drop(columns = 'RainTomorrow'))
y = df['RainTomorrow'] = np.where(df['RainTomorrow'] == 'Yes', 1, 0)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 6789)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

### Q2) train 데이터로 종속변수인 RainTomorrow(다음날의 강수 여부)를 예측하는 분류모델을 
###     3개 이상 생성하고 test 데이터에 대한 예측값을 csv파일로 각각 제출하시오.

In [ ]:
# !pip install xgboost

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
lr = LogisticRegression()
rf = RandomForestClassifier()
dt = DecisionTreeClassifier()
xgb = XGBClassifier()

In [ ]:
import datetime as dt

In [ ]:
X_train['Date'] = X_train['Date'].map(dt.datetime.toordinal)
X_test['Date'] = X_test['Date'].map(dt.datetime.toordinal)

In [ ]:
y_test

In [ ]:
lr.fit(X_train, y_train)
lr_y_pred = lr.predict(X_test)
# lr_y_pred.to_csv('lr_y_pred.csv')
# lr.score(lr_y_pred, y_test)

In [ ]:
pd.Series(lr_y_pred).value_counts()

In [ ]:
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)
# rf_y_pred.to_csv('rf_y_pred.csv')
# rf.score(rf_y_pred, y_test)

In [ ]:
pd.Series(rf_y_pred).value_counts()

In [ ]:
#y_train=np.where(rf_y_pred=='No', 0, 1)
y_train

In [ ]:
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)
# xgb_y_pred.to_csv('svc_y_pred.csv')
# xgb.score(svc_y_pred, y_test)

In [ ]:
pd.Series(xgb_y_pred).value_counts()

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, lr_y_pred)
print('오차 행렬:\n', confusion)

In [ ]:
lr_y_pred = pd.Series(lr_y_pred)
lr_y_pred.loc[lr_y_pred == 'Yes'] = 1
lr_y_pred.loc[lr_y_pred == 'No'] = 0
lr_y_pred = lr_y_pred.astype('int')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, lr_y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, rf_y_pred)
print('오차 행렬:\n', confusion)

In [ ]:
rf_y_pred = pd.Series(rf_y_pred)
rf_y_pred.loc[rf_y_pred == 'Yes'] = 1
rf_y_pred.loc[rf_y_pred == 'No'] = 0
rf_y_pred = rf_y_pred.astype('int')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf_y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, xgb_y_pred)
print('오차 행렬:\n', confusion)

In [ ]:
xgb_y_pred = pd.Series(xgb_y_pred)
xgb_y_pred.loc[xgb_y_pred == 'Yes'] = 1
xgb_y_pred.loc[xgb_y_pred == 'No'] = 0
xgb_y_pred = xgb_y_pred.astype('int')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, xgb_y_pred))

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
#y_test.loc[y_test == 'Yes'] = 1
#y_test.loc[y_test == 'No'] = 0

In [ ]:
y_test

In [ ]:
y_test = y_test.astype('int')

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, lr.decision_function(X_test))

plt.plot(fpr, tpr, label = 'ROC 곡선')
plt.xlabel('FPR')
plt.ylabel('TPR (재현율)')
# 0 근처의 임계값을 찾습니다.
close_zero = np.argmin(np.abs(thresholds))
plt.plot(fpr[close_zero], tpr[close_zero], 'o', markersize = 10,
        label = '임계값 0', fillstyle = 'none', c = 'k', mew = 2)
plt.legend(loc = 4)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, rf.predict_proba(X_test)[:, 1])

plt.plot(fpr, tpr, label = 'ROC 곡선')
plt.xlabel('FPR')
plt.ylabel('TPR (재현율)')
# 0 근처의 임계값을 찾습니다.
close_zero = np.argmin(np.abs(thresholds))
plt.plot(fpr[close_zero], tpr[close_zero], 'o', markersize = 10,
        label = '임계값 0', fillstyle = 'none', c = 'k', mew = 2)
plt.legend(loc = 4)

In [ ]:
from sklearn.metrics import roc_auc_score
rf_auc = roc_auc_score(y_test, rf.predict_proba(X_test)[:, 1])
# svc_auc = roc_auc_score(y_test, svc.decision_function(X_test))
print('랜덤 포레스트의 AUC: {:.3f}'.format(rf_auc))
# print('SVC의 AUC: {:.3f}'.format(svc_auc))

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, xgb.predict_proba(X_test)[:, 1])

plt.plot(fpr, tpr, label = 'ROC 곡선')
plt.xlabel('FPR')
plt.ylabel('TPR (재현율)')
# 0 근처의 임계값을 찾습니다.
close_zero = np.argmin(np.abs(thresholds))
plt.plot(fpr[close_zero], tpr[close_zero], 'o', markersize = 10,
        label = '임계값 0', fillstyle = 'none', c = 'k', mew = 2)
plt.legend(loc = 4)

In [ ]:
from sklearn.metrics import roc_auc_score
xgb_auc = roc_auc_score(y_test, xgb.predict_proba(X_test)[:, 1])
# svc_auc = roc_auc_score(y_test, svc.decision_function(X_test))
print('랜덤 포레스트의 AUC: {:.3f}'.format(xgb_auc))
# print('SVC의 AUC: {:.3f}'.format(svc_auc))

# 2. 통계분석 (사용 데이터 : bike_marketing)

### Q1) pop_density 변수를 factor형 변수로 변환하고, 
###     pop_density별 revenues의 평균 차이가 있는지 통계분석을 시행하여 결과를 해석하시오. 
###     만일 대립가설이 채택된다면 사후분석을 실시하고 결과를 해석하시오.

In [ ]:
df = pd.read_csv('./data/bike_marketing.csv')
print(df.shape)
df.head()

In [ ]:
df.info()

In [ ]:
df['pop_density'] = df['pop_density'].astype('category')

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

model = ols('revenues ~ C(pop_density)', df).fit()
anova_lm(model)

* 분산분석표를 통해 확인한 결과, SSA의 자유도는 2(집단의 수 -1 = 3-1), SST의 자유도는 169(관측값의 수 - 집단의 수 = 172-3)임을 확인할 수 있다. 분석 결과, p-value가 0.545로 유의수준 0.05하에서 대립가설을 기각한다. 따라서 세가지 인구밀집정도에 따른 매출은 통계적으로 유의한 차이를 가지고 있다고 보기 힘들다.

### Q2) google_adwords, facebook, twitter, marketing_total, employees가 
###     revenues에 영향을 미치는지 알아보는 회귀분석을 전진선택법을 사용하여 수행하고 결과를 해석하시오.

In [ ]:
model = sm.OLS(df['revenues'], df[['google_adwords', 'facebook', 'twitter', 'marketing_total', 'employees']]).fit()

In [ ]:
model.summary()

In [ ]:
model = sm.OLS.from_formula('revenues ~ google_adwords + facebook + twitter + marketing_total + employees', data = df).fit()
model.summary()

### Q3) 전진선택법을 사용해 변수를 선택한 후 새롭게 생성한 회귀모형에 대한 
###     잔차분석을 수행하고 결과를 해석하시오. 

In [ ]:
from statsmodels.stats.stattools import durbin_watson

#perform Durbin-Watson test
durbin_watson(model.resid)

* Durbin-Watson 검정 결과값이 2.11로 2에 가깝기 때문에 독립성 가정을 만족

In [ ]:
shapiro_test = stats.shapiro(model.resid)
shapiro_test

* Shapiro-Wilk 검정 결과 p-value가 0.09로 유의수준 0.05에서 대립가설을 기각한다. 따라서 bike 데이터는 정규분포를 따른다

* 잔차의 등분산성

In [ ]:
fitted = model.predict(df)
residual = df['revenues'] - fitted

In [ ]:
import seaborn as sns
import pandas as pd

a = [1, 2, 3]
b = [1, 2, 5]

zf=pd.DataFrame()
# df 데이터프레임에서 특정 열을 선택하여 변수에 할당
zf['a'] = np.array(a)
zf['b'] = np.array(b)
zf
sns.regplot(x='a', y='b', data = zf)

In [ ]:
zf = pd.DataFrame()
zf['x'] = fitted
zf['y'] = residual
sns.regplot(x='x',y='y', data = zf, lowess=True, line_kws={'color': 'red'})
plt.plot([fitted.min(), fitted.max()], [0, 0], '--', color='grey')

* 잔차 vs fitted value(예측값)의 분포(등분산성 가정 확인)
* 그래프의 기울기를 나타내는 빨간색 선이 포물선의 성향을 띠고 있기 때문에 잔차는 평균인 0을 중심으로 고르게 분포한다고 판단하기 어렵다

## qqplot(정규성 가정 확인)

In [ ]:
from statsmodels.graphics.gofplots import qqplot
qqplot(model.resid, line='s')
plt.show()

* qqplot을 확인결과, 오른쪽 위를 향하는 45도 각도의 직선위에 그래프의 점들이 주로 위치한 것으로 보아 bike의 데이터가 정규성을 만족

# Scale_Location(등분산성 가정 확인)

In [ ]:
sr = stats.zscore(residual)

In [ ]:
xf = pd.DataFrame()
xf['x'] = fitted
xf['y'] = np.sqrt(np.abs(sr))

sns.regplot(x='x', y='y', data=xf, lowess=True, line_kws={'color': 'red'})

* 빨간선의 기울기가 0에 가까워야 하지만, fitted_value가 커질수록 기울기가 다소 변화하는 경향을 보이고있다. 이렇게 빨간선의 기울기가 0에서 떨어진 점이 있다면 해당 점에서는 표준화 잔차가 큼을 의미하고, 회귀 직선이 y값을 잘 적합하지 못함을 의미한다. 또한 해당 점들은 이상치일 가능성이 있다.

# 극단값

In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence

In [ ]:
cd, _ = OLSInfluence(model).cooks_distance

In [ ]:
cd.sort_values(ascending=False).head()

# 3. 비정형 데이터마이닝 (사용 데이터 : "instagram_태교여행")

### Q1) ‘instagram_태교여행.txt’ 데이터를 읽어온 뒤 숫자, 특수 문자 등을 
###     제거하는 전처리 작업을 시행하시오.

In [ ]:
def read_data(filename):
    with open(filename, 'r', encoding = 'cp949') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

In [ ]:
text = read_data('./data/instagram_태교여행.txt')

In [ ]:
' '.join(x for x in text)

In [ ]:
text = ' '.join(x for x in np.array(text).reshape(-1,))

In [ ]:
import konlpy
import re

In [ ]:
text = re.sub(r'\d+', '', text)

In [ ]:
text[:1000]

In [ ]:
def cleansing(text):
    repl = ''
    pattern  = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
    text = re.sub(pattern = pattern, repl = repl, string = text)
    pattern  = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern = pattern, repl = repl, string = text)
    pattern  = '([ㄱ-ㅎㅏ-ㅣ]+)'
    text = re.sub(pattern = pattern, repl = repl, string = text)
    pattern  = '<[^>]*>'
    text = re.sub(pattern = pattern, repl = repl, string = text)
    pattern = '[^\w\s]'
    text = re.sub(pattern = pattern, repl = repl, string = text)
    return text    

In [ ]:
text = cleansing(text)

### Q2) 전처리된 데이터에서 “태교여행”이란 단어를 사전에 추가하고 명사를 추출해 
###     출현빈도 10위까지 막대그래프로 시각화하시오. 

In [ ]:
taegyo = pd.DataFrame(columns = ['명사', '형태소'])
taegyo['명사'] = ['태교여행']
taegyo['형태소'] = ['NNP']

In [ ]:
taegyo

In [ ]:
taegyo.to_csv('./taegyo.txt', sep='\t', index=False, header=False)

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab('C:/mecab/mecab-ko-dic')
nouns = mecab.nouns(text)

In [ ]:
from collections import Counter
cnt = Counter(nouns)

In [ ]:
temp_df = pd.DataFrame(columns = ['명사', '빈도'])
freq = cnt.most_common(10)
temp_list = []
temp_list2= []
for i in freq:
    temp_list.append(i[0])
    temp_list2.append(i[1])
temp_df['명사'] = temp_list
temp_df['빈도'] = temp_list2
temp_df

In [ ]:
plt.figure(figsize = (8,6))
sns.barplot(x = '명사', y = '빈도', data = temp_df)
plt.tight_layout()
plt.show()

In [ ]:
words = {}
for i in cnt.most_common(20):
    words[i[0]] = i[1]

In [ ]:
from wordcloud import WordCloud 
import nltk
from nltk.corpus import stopwords

wordcloud = WordCloud(font_path = "NanumGothicCoding.ttf", background_color='white',colormap = "Accent_r",
                      width=1500, height=1000).generate_from_frequencies(words)

plt.imshow(wordcloud)
plt.axis('off')
plt.show()